# **CIFAR10**

In [ ]:
import torch
import torch.nn as nn 
import torch.optim as optim
import torchvision
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()  # Check if GPU available 
print("Use GPU?", use_cuda)

class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # change the input channels from 1 to 3
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.avg_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.avg_pool2d(out, 2)
        # out.size() = [batch_size, channels, size, size], -1 here means channels*size*size
        # out.view(out.size(0), -1) is similar to out.reshape(out.size(0), -1), but more efficient
        # Think about why we need to reshape the out?
        out = out.view(out.size(0), -1) 
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out
 

my_model = model()

if use_cuda:
  my_model = my_model.cuda()  # transfer model to GPU

minibatch_size = 128
criterion = nn.CrossEntropyLoss()


def adjust_learning_rate(optimizer, num_epochs, init_lr):
    #lr = 1.0 / (epoch + 1) 
    lr = init_lr * 0.1 ** (num_epochs // 10)  
    # epoch // 10, calculate the quotient 
    # divide lr by 10 every 10 epoches
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

init_lr = 0.1
num_epochs = 30
optimizer = optim.SGD(my_model.parameters(), lr=init_lr, weight_decay = 0.0001)


# Step 2: Define a loss function and training algorithm
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(my_model.parameters(), lr=lr)


# Step 3: load dataset

CIFAR10_transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=CIFAR10_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=minibatch_size, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=CIFAR10_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

# classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


#Step 4: Train the NNs
# One epoch is when an entire dataset is passed through the neural network only once.

for epoch in range(num_epochs):
    current_lr = adjust_learning_rate(optimizer, epoch, init_lr)
    for i, (images, labels) in enumerate(trainloader):
        if use_cuda:
          images = images.cuda()
          labels = labels.cuda()

        # Forward pass to get the loss
        outputs = my_model(images) 
        loss = criterion(outputs, labels)
        
        # Backward and compute the gradient
        optimizer.zero_grad()
        loss.backward()  #backpropragation
        optimizer.step() #update the weights/parameters
        
    # Training accuracy
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(trainloader):
        if use_cuda:
          images = images.cuda()
          labels = labels.cuda()  
        outputs = my_model(images)
        p_max, predicted = torch.max(outputs, 1) 
        total += labels.size(0)
        correct += (predicted == labels).sum()
    training_accuracy = float(correct)/total

    
    # Test accuracy
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(testloader):
        if use_cuda:
          images = images.cuda()
          labels = labels.cuda()
        outputs = my_model(images)
        p_max, predicted = torch.max(outputs, 1) 
        total += labels.size(0)
        correct += (predicted == labels).sum()
    test_accuracy = float(correct)/total
        
    print('Epoch: {}, Training accuracy: {}, Test accuracy: {}' .format(epoch+1,training_accuracy,test_accuracy))  

Use GPU? True
Files already downloaded and verified
Files already downloaded and verified
Epoch: 1, Training accuracy: 0.36078, Test accuracy: 0.3535
Epoch: 2, Training accuracy: 0.49602, Test accuracy: 0.4949
Epoch: 3, Training accuracy: 0.51566, Test accuracy: 0.5046
Epoch: 4, Training accuracy: 0.52948, Test accuracy: 0.5114
Epoch: 5, Training accuracy: 0.54602, Test accuracy: 0.5238
Epoch: 6, Training accuracy: 0.4967, Test accuracy: 0.4824
Epoch: 7, Training accuracy: 0.60518, Test accuracy: 0.5662
Epoch: 8, Training accuracy: 0.59802, Test accuracy: 0.554
Epoch: 9, Training accuracy: 0.61008, Test accuracy: 0.5623
Epoch: 10, Training accuracy: 0.66042, Test accuracy: 0.5933
Epoch: 11, Training accuracy: 0.6765, Test accuracy: 0.5975
Epoch: 12, Training accuracy: 0.6792, Test accuracy: 0.5907
Epoch: 13, Training accuracy: 0.6833, Test accuracy: 0.6007
Epoch: 14, Training accuracy: 0.69032, Test accuracy: 0.5987
Epoch: 15, Training accuracy: 0.71752, Test accuracy: 0.6077
Epoch: 16

# **MNIST**

In [ ]:
import torch
import torch.nn as nn 
import torch.optim as optim
import torchvision
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()
print('Use GPU?', use_cuda)

# Define a LeNet-5
# Note that we need to reshape MNIST imgaes 28*28 to 32*32
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.avg_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1) 
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

my_model = model()

if use_cuda:
  my_model = my_model.cuda()  # transfer model to GPU

minibatch_size = 128
criterion = nn.CrossEntropyLoss()


def adjust_learning_rate(optimizer, num_epochs, init_lr):
    #lr = 1.0 / (epoch + 1) 
    lr = init_lr * 0.1 ** (num_epochs // 10)  
    # epoch // 10, calculate the quotient 
    # divide lr by 10 every 10 epoches
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

init_lr = 0.1
num_epochs = 20
optimizer = optim.SGD(my_model.parameters(), lr=init_lr, weight_decay = 0.0001)


# Step 3: load dataset

MNIST_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((32, 32)),
                                                  torchvision.transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train= True, download=True, transform=MNIST_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=minibatch_size)

testset = torchvision.datasets.MNIST(root='./data', train= False, download=True, transform=MNIST_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=len(testset)) 

for epoch in range(num_epochs):
    
    for i, (images, labels) in enumerate(trainloader):
        if use_cuda:
          images = images.cuda()
          labels = labels.cuda()

        # Forward pass to get the loss
        outputs = my_model(images) 
        loss = criterion(outputs, labels)
        
        # Backward and compute the gradient
        optimizer.zero_grad()
        loss.backward()  #backpropragation
        optimizer.step() #update the weights/parameters
        
    # Training accuracy
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(trainloader):
        if use_cuda:
          images = images.cuda()
          labels = labels.cuda()  
        outputs = my_model(images)
        p_max, predicted = torch.max(outputs, 1) 
        total += labels.size(0)
        correct += (predicted == labels).sum()
    training_accuracy = float(correct)/total

    
    # Test accuracy
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(testloader):
        if use_cuda:
          images = images.cuda()
          labels = labels.cuda()
        outputs = my_model(images)
        p_max, predicted = torch.max(outputs, 1) 
        total += labels.size(0)
        correct += (predicted == labels).sum()
    test_accuracy = float(correct)/total
        
    print('Epoch: {}, Training accuracy: {}, Test accuracy: {}' .format(epoch+1,training_accuracy,test_accuracy))  

Use GPU? True
Epoch: 1, Training accuracy: 0.8906833333333334, Test accuracy: 0.8943
Epoch: 2, Training accuracy: 0.9568333333333333, Test accuracy: 0.9604
Epoch: 3, Training accuracy: 0.9698333333333333, Test accuracy: 0.9707
Epoch: 4, Training accuracy: 0.9751166666666666, Test accuracy: 0.9743
Epoch: 5, Training accuracy: 0.979, Test accuracy: 0.9767
Epoch: 6, Training accuracy: 0.98145, Test accuracy: 0.9795
Epoch: 7, Training accuracy: 0.9836666666666667, Test accuracy: 0.98
Epoch: 8, Training accuracy: 0.9857666666666667, Test accuracy: 0.9824
Epoch: 9, Training accuracy: 0.9874, Test accuracy: 0.9834
Epoch: 10, Training accuracy: 0.9883833333333333, Test accuracy: 0.9839
Epoch: 11, Training accuracy: 0.98945, Test accuracy: 0.9849
Epoch: 12, Training accuracy: 0.98915, Test accuracy: 0.9842
Epoch: 13, Training accuracy: 0.9906833333333334, Test accuracy: 0.9853
Epoch: 14, Training accuracy: 0.9910666666666667, Test accuracy: 0.9844
Epoch: 15, Training accuracy: 0.990533333333333